# Trendyol Data Science Bootcamp - You Do #2

## Import ve datayı hazırlama

In [97]:
import pandas as pd
import numpy as np
from tqdm import trange

In [98]:
df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',
                 names=['user_id', 'item_id', 'rating', 'timestamp'])
r = df.pivot(index='user_id', columns='item_id', values='rating').values

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [99]:
# r = np.array([[7, 6, 7, 4, 5, 4],
#               [6, 7, np.nan, 4, 3, 4],
#               [np.nan, 3, 3, 1, 1, np.nan],
#               [1, 2, 3, 3, 3, 4],
#               [1, np.nan, 1, 2, 3, 3]])

In [100]:
r

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 5., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  5., nan, ..., nan, nan, nan]])

In [101]:
irow, jcol = np.where(~np.isnan(r))

idx = np.random.choice(np.arange(100_000), 1000, replace=False)
# idx = np.random.choice(np.arange(5), 3, replace=False)

test_irow = irow[idx]
test_jcol = jcol[idx]

In [102]:
r_train = r.copy()
r_train[test_irow, test_jcol] = np.nan

## Part 1 - Gradient Descent

In [103]:
def gradient_descent(r: np.array, alpha=0.001, max_iteration=500, early_stopping=-1):
  # epsilon = 0.1
  m, n = r.shape
  b_user, b_item = np.random.rand(m), np.random.rand(n)
  
  t = trange(max_iteration)
  for _ in t:
    loss = 0
    grad_b_user, grad_b_item = np.zeros(m), np.zeros(n)

    for u in range(r.shape[0]):
      for j in range(r.shape[1]):
        if np.isnan(r[u, j]):
          continue
        y = r[u, j]
        y_hat = b_user[u] + b_item[j]

        grad_b_user[u] -= (y - y_hat)
        grad_b_item[j] -= (y - y_hat)

        loss +=  ((y - y_hat) ** 2) / 2

    # IMPLEMENT EARLY STOPPING LOGIC
    
    b_user -= alpha * grad_b_user
    b_item -= alpha * grad_b_item

    t.set_description(f'Loss: {loss:.2f}\t')

  return b_user, b_item

In [105]:
b_user, b_item = gradient_descent(r_train, max_iteration=100)

Loss: 41918.26	: 100%|██████████| 100/100 [06:20<00:00,  3.80s/it]


In [106]:
r_hat = np.zeros(r.shape)

In [107]:
for u in range(r.shape[0]):
      for j in range(r.shape[1]):
        r_hat[u, j] = b_user[u] + b_item[j]

In [108]:
r

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 5., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  5., nan, ..., nan, nan, nan]])

In [121]:
np.around(r_hat, 2)

array([[3.96, 3.3 , 3.12, ..., 2.16, 2.06, 2.6 ],
       [3.96, 3.3 , 3.12, ..., 2.16, 2.06, 2.59],
       [3.33, 2.68, 2.49, ..., 1.53, 1.43, 1.97],
       ...,
       [4.03, 3.37, 3.19, ..., 2.23, 2.13, 2.67],
       [4.36, 3.71, 3.52, ..., 2.56, 2.46, 3.  ],
       [3.85, 3.2 , 3.01, ..., 2.06, 1.96, 2.49]])

## Part 2 - Work in Progress